# ChIP-Seq Project: The Case of The Mixed-up Sample Labels 

## _Investigating Sequencing depth and Histone Methylation Marks_

The following tasks have been adapted by Vivek Iyer and Victoria Offord from materials developed by Angela Goncalves, Myrto Kostadima, Steven Wilder and Maria Xenophontos.

# An experiment, some sequence ...

You are your lab's only informatician this week. Late on a Friday afternoon, you receive a distressed call from your PI. Earlier this week, she received ENCODE sample GM12878 in the mail. She split the sample into three vials:

* the first vial was used to do ChipSeq to detect H3k4me3. The sequence produced was called  "H3k4me3.bam" by the sequencing facility.
* the second vial was used to do ChipSeq to detect H3k27me3. The sequence produced was called "H3k27me3.bam" by the sequencing facility.
* the third vial was RNASeq'd to detect gene expression. The RNASeq pipeline produced sequence called "genes.fpkm.txt".

Upon hearing of this experiment, your PI's funding agency called her up to ask for 

* the bam & expression files immediately. The correctly labelled files have to be delivered this afternoon.
* an analysis of whether your experiment could be done with less sequencing, without affecting the results

They want answers by the end of the day! Any delay will probably cost your PI her grant - and with it, your position.

# And then ... disaster

You overheard your lab manager and your PI having a loud argument in her office this morning. Your lab manager then abruptly quit. This wasn't a surprise - they had been at loggerheads for months.

The real surprise was that - just before quitting - he randomly re-assigned the labels of the H3k4me3 and H3k27me3 ChipSeq bam files, calling them "v1.bam" and "v2.bam". He then caught the next flight to Sydney. He'll be out of touch a while.

# The challenge

Your lab have to deliver the sequence files to the funding agency in 2.5 hours, but you have no idea whether:
* v1.bam = H3k4me3.bam and v2.bam = H3k27me3.bam

**...OR...** 

* v1.bam = H3k27me3.bam and v2.bam = H3k4me3.bam

AND

* you haven't done the analysis of whether you can use less sequencing

If you can't work all this out and deliver the correctly labelled sequence files, your lab's funding will be cut off, so a lot is riding on this analysis. Good luck!

## Chromatin Environment

The goal of this hands-on session is to investigate the chromatin environment around gene features and regulatory elements. We will be plotting where the different histone modifications occur in relation to genes.

### Data and software

Your goal is to investigate the chromatin environment around gene features and regulatory elements. You will be plotting where the different histone modifications occur in relation to genes.

Your lab investigated histone modifications from ENCODE sample GM12878. The BAM alignment files - after they were anonymised by the lab manager - have been labelled 'v1.bam' and 'v2.bam'. 

_(... ok, ok, in reality they were actually downloaded from http://www.encodeproject.org. All ENCODE experiments had at least 2 technical replicates, but we will only be using the first replicate for simplicity.)_

You need to use **ngsplot** https://github.com/shenlab-sinai/ngsplot to visualise these datasets at transcription start sites and inside gene bodies. The ngsplot database for the human gene set has been generated from the Ensembl http://www.ensembl.org and RefSeq http://www.ncbi.nlm.nih.gov/refseq/ gene sets, by default using the Ensembl set.

In [ ]:
ngs.plot.r

Further help on the ngsplot options can be found at https://github.com/shenlab-sinai/ngsplot/wiki/ProgramArguments101.

## Histone Marks ##

The histone modifications we will be looking at today are all thought to play different roles in the regulation of gene expression and the putative functions. The ENCODE Project summary is in the picture below (Figure 1). 

The histone modifications are named by the histone tail (e.g. H3), location in the protein sequence (e.g. K27) and the biochemical modification (e.g. me3). For instance, H3K4me3 refers to the trimethylation of the lysine (K) in the fourth position in the protein sequence of histone 3 tail. 

Notice that although residues K,R,E,S,T and P could be modified by Acetylation, SUMOlyation etc, we will focus on two well-understood marks, H3k4me3 and H3k27me3.

Can you spot H3k4me3 and H3k27me3 in Figure 2, as well as their putative functions? You can check this against the first sections of these articles: https://en.wikipedia.org/wiki/H3K4me3 and https://en.wikipedia.org/wiki/H3K27me3. 

![figure 1](images/figure1.png)

_Figure 1: The histone modifications are named by the histone tail, location in the protein sequence and the biochemical modification, e.g. H3K4me3, refers to the trimethylation of the lysine (K) in the fourth position in the protein sequence of histone 3 tail._

![figure 2](images/figure2.png)

_Figure 2: A summary of the putative functions of various histone marks, according to the ENCODE Project._

### Your strategy to fill in the missing labels ###

**Question1:** Based on the figures and web pages above, fill in these sentences:

* H3K4me3 is a ______ mark, typically associated with the ______ of genes. 
* H3k27me3 is a ______ mark, associated with the ______ of genes.

You have two sequence files: v1.bam and v2.bam. If you could establish 
the location of the peaks of v1.bam and v2.bam independently high-expressed 
and low-expressed genes you could compare with these expectations,and so 
identify the labels. 

_This is where ngs.plot.r comes in._ **ngs.plot.r is not a ChipSEQ peak-finder like macs2**. Rather, 
* it 'collapses' together all the genomic segments around sets of genes in a standardised way
* it indicates how many reads are 'piled up' on the standardised genomic segment

The result is an indication of how much _sequence_ is piled up on average at the TSS 
and in gene bodies, either for all genes, or for subsets of genes. We can use
these indications to _infer_ where the actual ChipSeq peaks are concentrated.

### Test run: Transcription Start Sites for v2.bam

Let's give this a go! We will start by looking at the genome-wide profiles for _one_ of our histone marks, just around the TSS, for every gene in the genome.

**Plot the distribution of v2.bam reads around protein-coding gene TSS (this may take a few minutes to run).**

In [ ]:
ngs.plot.r -G hg19 -O v2.tss -FL 150 -R tss \
-C v2.bam -T v2

**Question2:** You can now view the v2 aggregation plots and heatmaps in your folder. Look at the picture v2.tss.avgprof.pdf. Where does the majority of the v2 signal appear in relation to TSS?

**Question3:** Look at v2.tss.heatmap.pdf. This heatmap shows the read-count size 
(heatmap colour) for every gene (y-axis), where the x-axis is the location relative to the TSS. Is the average profile you see in v2.tss.avgprof.pdf actually the case for _every_ gene? If not, roughly what proportion of genes display this pattern?


**Modify this analysis to plot read distribution around genebody, taking control into account.**

ngsplot also enables you to correct the ChIP sample using the control sample. This aims to correct for any genome biases in alignability, GC content etc. Further, ngsplot will show the averaged read distribution around _gene bodies_ where all genes have been mapped onto a standard genomic interval.

**Let's give this a try for v2.bam**

In [ ]:
ngs.plot.r -G hg19 -O v2.control.genebody -FL 150 -R genebody \
-C v2.bam:Control.bam -T v2:control

### Gene Expression

You will now use RNA-seq data form the same cell type results to separate all of the Ensembl genes into three categories, based on the expression (FPKM values) of the genes. The file containing the FPKMs for GM12878 whole cell RNA-seq is called `genes.fpkm_tracking`. These FPKM values have been computed by Cufflinks.

**Read the first few lines of the GM12878 whole cell genes.fpkm_tracking file.**

In [ ]:
head -n 10 genes.fpkm_tracking

**Remove the header line.**

In [ ]:
sed '1d' genes.fpkm_tracking > genes.fpkm.txt

**Sort the genes by FPKM score.**

In [ ]:
sort -k10 -n -r genes.fpkm.txt > genes.fpkm.sorted.txt

**Count the number of genes.**

In [ ]:
wc -l genes.fpkm.sorted.txt

**Use `head`, `sed` and `awk` to pull out the Highest, Lowest and Intermediate expression genes.**

In [ ]:
head -n 5000 genes.fpkm.sorted.txt | \
awk '{print $1}' > high_expressed_genes.txt

In [ ]:
awk '{ if($10 ==0) {print $1} }' \
genes.fpkm.sorted.txt > low_expressed_genes.txt

In [ ]:
sed -n '10001,15000p' genes.fpkm.sorted.txt | \
awk '{print $1}' > mid_expressed_genes.txt

**Create a tab-delimited configuration file, "expression_v2.txt", in a text editor: it should contain the following lines:**

    |--------------|--------------------------|--------|
    | v2.bam       | high_expressed_genes.txt | "High" |
    | v2.bam       | mid_expressed_genes.txt  | "Med"  |
    | v2.bam       | low_expressed_genes.txt  | "Low"  |

**Then run the following command to plot the average read depth etc for these three groups of genes, in the same plot:**

In [ ]:
ngs.plot.r -G hg19 -R genebody -C expression_v2.txt \
-O v2.express.genebody -D ensembl \
-FL 300 -T v2.expression

**Question 4: What patterns can you see for the read depths for the different gene groups?**

Notice that the average read-level level for v2.bam over the gene body is quite different for those genes which are known to be highly expressed (as measured by RNASeq) in the sample, compared to those mid- or low-expressed. 

# Read depth profile vs. Gene expression for v1.bam

Repeat the same read-depth analysis for v1.bam: your goal is to see the differences in read-depth profile for the experiment in v1.bam, for high- mid- and low-expressed genes. First, create a file "expression_v1.txt" by copying "expression_v2.txt" and modifying it, altering "v2" to "v1"

In [ ]:
cp expression_v2.txt expression_v1.txt

# manually edit expression_v1.txt to alter 'v2' to 'v1'

Now run ngs.plot.r to find read depth profiles for v1.bam, for different gene expression groups:

In [ ]:
ngs.plot.r -G hg19 -R genebody -C expression_v1.txt \
-O v1.express.genebody -D ensembl \
-FL 300 -T v1.expression

# Look at your data and form a conclusion

You now have two read-depth vs gene-expression profiles: 

* v1.express.genebody.R.pdf
* v2.express.genebody.R.pdf

### Question 5: which of these has high read-depths in high-expressed genes?
### Question 6: which of these has high read-depths in low-expressed genes?
### Question 7: by comparing these patterns to what you know about H3k4me3 and H3k27me3 modifications, can you conclude whether v1.bam is actually H3k4me3 or H3k27me3?

## Sequencing depth

One of the most frequent questions that come up in ChIP-seq experiments is whether the sequencing depth is sufficient.

The more we sequence a ChIP-seq library, the more peaks of low fold change we will identify. Therefore, the only way to answer that question is to look for the number of peaks identified when we down sample our library. To test for sufficient sequencing depth in our sample we will down sample our ChIP and Control datasets to 10%, 20%, .., 90% of the initial library size and call peaks. To do so, we will use the functions **randsample** and **callpeak** from macs2, respectively.

**First, go to the group_projects folder.**

In [ ]:
cd /home/manager/course_data/group_projects

**Check to see if the ChIPSeq-Project1 folder exists.**

In [ ]:
ls ChIPSeq-Project1

**If this folder doesn't exist, please check with your course instructor.**

**Once you have the data, go into the ChIPSeq-Project1 directory.**

In [ ]:
cd ChIPSeq-Project1

Below are a series of commands which will downsample the ChIP and Control datasets to 10%, 20%, .., 90% of the initial library size and call peaks. To do this we use a while loop which will start counting from 10 (`i=10`), in intervals of 10 (e.g. 10, 20, 30...) (`((i = i + 10))`) up to 100 (`$i -lt 100`). The value which is being counted is stored in a variable called `i` that is then referenced in other commands using `$i`. Finally, to make sure we know where we are in the loop, we will print a message in the terminal `echo "Looking at ${i}% of the reads"`.

The command we are using to downsample the ChIP and Control datasets reads is `macs2 randsample` which we will be giving an input file `-t`, a percentage `-p`, an output file `-o`, an output directory `--outdir` and the format of the input file `-f`.

**Look at the usage for `macs2 randsample`.**

In [ ]:
macs2 randsample -h

We then call the peaks for each set of downsampled reads using `macs2 callpeak` in a similar way to the ChIP-Seq tutorial.

**Type the commands below into a file called `project1.sh`.**

In [ ]:
i=10

while [[ $i -lt 100 ]]
do

echo "Looking at ${i}% of the reads"

macs2 randsample -t PAX5.bam -p $i -o PAX5.perc${i}.bed \
--outdir macs2_downsample -f BAM

macs2 randsample -t Control.bam -p $i -o Control.perc${i}.bed \
--outdir macs2_downsample -f BAM

macs2 callpeak -t macs2_downsample/PAX5.perc${i}.bed \
-c macs2_downsample/Control.perc${i}.bed --gsize 138000000 \
--format BED --name macs2_downsample/PAX5.perc${i} \
--pvalue 1e-3 --call-summits

((i = i + 10))

done

**Now, make your script executable.**

In [ ]:
chmod u+x project1.sh

**And run the script.**

In [ ]:
./project1.sh

Next, we want to plot the number of peaks called in each of the downsampled datasets. To do this we will use `R`.  First we tell R where to find our output files using `setwd`. We then use a for loop to extract only the peaks whose overall enrichment meets a threshold value (`fc.thres <- 4` and `peaks <- peaks[peaks[, 7] > fc.thres, ]`). We then count those peaks (`no.peaks <- c(no.peaks, nrow(peaks))`). Finally, we plot the number of peaks vs the percentage, saving the output to `peaks_vs_percentage.jpg`.

**Type the following commands into a file called `script.R`.**

In [ ]:
rm(list = ls())

options(stringsAsFactors=F)
setwd( "/home/manager/course_data/group_projects/ChIPSeq-Project1" )

fc.thres <- 4

no.peaks <- c()
for(row in seq(from=10, to = 90, by = 10))
{
	print(row)
    
	peaks <- read.table(paste("macs2_downsample/PAX5.perc", 
    row, "_peaks.narrowPeak", sep=""))
    
	peaks <- peaks[peaks[, 7] > fc.thres, ]

	no.peaks <- c(no.peaks, nrow(peaks))
}

peaks <- read.table("PAX5_peaks.narrowPeak")
peaks <- peaks[peaks[, 7] > fc.thres, ]
no.peaks <- c(no.peaks, nrow(peaks))


jpeg('peaks_vs_percentage.jpg')

plot(seq(from=10, to = 100, by = 10), no.peaks, 
type="o", col="blue", xlab="Percentage of reads", ylab="Number of peaks")

dev.off()

**Use `Rscript` to run your R script.**

In [ ]:
Rscript script.R

**Let's take a look at the plot.**

In [ ]:
eog peaks_vs_percentage.jpg

**Q1: Do you think that we have sequenced enough?**

Close the file when you have finished looking at the plot.